# Assignment 3 Dashboard
**Name: Rommel Antunez Barrios**

**e-mail: rommel.antunez6474@alumnos.udg.mx**

# Modules

In [12]:
import numpy as np
from scipy.stats import wrapcauchy
from matplotlib import pyplot as plt
from scipy.spatial import distance
import math

from scipy.stats import levy_stable

# from plotly
import plotly.graph_objects as go

# from pandas
import pandas as pd

# from panel
import panel as pn
pn.extension("plotly", sizing_mode="stretch_width", template="fast") # this is needed to use plotly in panel

# Functions and classes

## Vec2d

In [13]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## Brownian Motion

In [14]:
def brownian_motion(speed, n_steps, initial_x, initial_y):
    # Init velocity vector
    velocity = Vec2d(speed,1)

    # Init empty DF
    BM_2d_df = pd.DataFrame(columns=['x_pos','y_pos', 'z_pos'])
    # Add starting position to DF
    temp_df = pd.DataFrame([{'x_pos':initial_x, 'y_pos':initial_y, 'z_pos':0}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True) 
    # Generate trajectory
    for i in range(n_steps-1):
        turn_angle = np.random.choice([0, np.pi/2, np.pi, 3*np.pi/2])
        # We are rotating the velocity vector
        velocity = velocity.rotated(turn_angle)

        # Update to new position
        temp_df = pd.DataFrame([{
                'x_pos':BM_2d_df.x_pos[i] + velocity.x, 
                'y_pos':BM_2d_df.y_pos[i] + velocity.y,
                'z_pos':BM_2d_df.z_pos[i] + velocity.get_length()
            }])
        BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True) 
    return BM_2d_df

## Levy Flight

In [15]:
def levy_flight(speed=5, n_steps=1000, initial_x=0, initial_y=0,cauchy_coefficient=1.2):
    # Initial variables
    time_per_step=0.0001
    max_step_length=10
    s_pos = [initial_x, initial_y, 0]
    beta = 0
    m = 1

    trajectory = [{'x_pos': s_pos[0], 'y_pos': s_pos[1], 'z_pos': s_pos[2]}]

    # Implementation of Vec2d class
    velocity = Vec2d(speed, 0)

    rotations = levy_stable.rvs(cauchy_coefficient, beta, loc=m, scale=1, size=n_steps)

    for i in range(n_steps):
        turn_angle = rotations[i]  # Get rotations from levy distribution.
        step_length = levy_stable.rvs(cauchy_coefficient, beta)
        step_length = min(step_length, max_step_length)
        velocity = velocity.rotated(turn_angle)  # Use rotated function from Vec2d class

        # Dict, later to be appended to list
        new_position = {
            'x_pos': trajectory[i]['x_pos'] + velocity.x * float(step_length),
            'y_pos': trajectory[i]['y_pos'] + velocity.y * float(step_length),
            'z_pos': trajectory[i]['z_pos'] + velocity.get_length() * time_per_step * float(step_length)  # Calculated distance
        }
        # List with all the trajectory position
        trajectory.append(new_position)

    # Create Pandas Data Frame with trajectories
    levy_pdf = pd.DataFrame(trajectory)
    return levy_pdf


## Correlated Random Walk

In [16]:
def correlated_random_walk(speed=5, n_steps=1000, initial_x=0, initial_y=0, cauchy_coefficient=0.5):
    # Initial variables
    s_pos = [initial_x, initial_y, 0]
    CRW = np.ones(shape=(n_steps, 3)) * s_pos
    
    if cauchy_coefficient >= 1.0:
        cauchy_coefficient = 0.99999
    
    # Implement a Vec2d class vector
    velocity = Vec2d(speed, 0)
    
    # Selecting rotation from cauchy distribution
    rotations = wrapcauchy.rvs(cauchy_coefficient, loc=0, size=n_steps)
    
    for i in range(1, n_steps):
        turn_angle = np.random.choice(rotations)
        # Rotate the velocity vector
        velocity = velocity.rotated(turn_angle)
        # Update to new position
        CRW[i, 0] = CRW[i-1, 0] + velocity.x
        CRW[i, 1] = CRW[i-1, 1] + velocity.y
        CRW[i, 2] = CRW[i-1, 2] + velocity.get_length()
    
    # Create a pandas DataFrame for CRW
    df_CRW = pd.DataFrame(CRW, columns=['x_pos', 'y_pos', 'z_pos'])
    return df_CRW

## Mean Squared Displacement

In [17]:
def mean_squared_displacement(trajectory):
    size = len(trajectory)-1
    msd = []
    for i in range(1, size):
        msd.append(np.mean((trajectory['x_pos'].iloc[i:].values - trajectory['x_pos'].iloc[:-i].values) ** 2 + (trajectory['y_pos'].iloc[i:].values -  trajectory['y_pos'].iloc[:-i].values) ** 2 ))

    return pd.DataFrame(msd, columns=['y'])

## Path Lenght

In [18]:
def path_length(trajectory):
    # Init variables
    path_length = 0
    pl_trajectory = []
    # Calculate path length
    for i in range(len(trajectory)-1):
        path_length += distance.euclidean([trajectory.x_pos[i], trajectory.y_pos[i]], [trajectory.x_pos[i+1], trajectory.y_pos[i+1]])
        pl_trajectory.append(path_length)
    return  pd.DataFrame(pl_trajectory, columns=['y'])

## Turning Angle Distribution

In [19]:
def cross2d(x, y):
    return x[..., 0] * y[..., 1] - x[..., 1] * y[..., 0]

def turning_angles(trajectory):
    turning_angles = []
    for i in range(1, len(trajectory)-1):
        # Calculate the turning angle
        a = np.array([trajectory.x_pos[i-1], trajectory.y_pos[i-1]])
        b = np.array([trajectory.x_pos[i], trajectory.y_pos[i]])
        c = np.array([trajectory.x_pos[i+1], trajectory.y_pos[i+1]])
        ab = b - a
        bc = c - b
        dot_product = np.dot(ab, bc)
        cross_p = cross2d(ab, bc)
        orient = np.sign(cross_p)
        if orient == 0:
            orient = 1
        denominator = np.linalg.norm(ab) * np.linalg.norm(bc) + np.finfo(float).eps
        angle = np.arccos(np.clip((dot_product / denominator), -1.0, 1.0)) # apply the arccos function to the dot product to get the angle without cos 
        new_angle = angle * orient
        turning_angles.append(round(new_angle, 2))
    return pd.DataFrame(turning_angles, columns=['y'])

# Dashboard

In [20]:
# ----------------- CREATE WIDGETS -----------------


# Create radio buttons for filter selection
trajectory_filter_radio = pn.widgets.RadioButtonGroup(
    name='Filter', 
    options=['BM', 'CRW', 'LF'],
    value='BM'
)

metric_filter_radio = pn.widgets.RadioButtonGroup(
    name='Metric',
    options=['PL', 'MSD', 'TAD'],
    value='PL'
)

# Create widgets for number of steps, speed, and initial positions
steps_slider = pn.widgets.IntSlider(name='Number of Steps', start=50, end=1000, step=50, value=50)
speed_slider = pn.widgets.IntSlider(name='Speed', start=1, end=12, step=1, value=1)
init_x_input = pn.widgets.IntInput(name='Init X Position', value=0)
init_y_input = pn.widgets.IntInput(name='Init Y Position', value=0)

# Create widget for cauchy coefficient, initially hidden
cauchy_coeff_input = pn.widgets.FloatInput(name='Cauchy Coefficient / Alpha', value=0.5, visible=False, step=0.05, start=0.1, end=0.9999)

# ----------------- FUNCTION -----------------

# Function to show/hide cauchy coefficient input based on filter selection
def update_cauchy_coeff(event):
    if trajectory_filter_radio.value in ['CRW', 'LF']:
        cauchy_coeff_input.visible = True
    else:
        cauchy_coeff_input.visible = False

# Attach the update function to the filter radio button group
trajectory_filter_radio.param.watch(update_cauchy_coeff, 'value')

# ----------------- PLOT 3D TRAJECTORY -----------------

# Plot Different Trajectories
@pn.depends(steps_slider, speed_slider, init_x_input, init_y_input, cauchy_coeff_input, trajectory_filter_radio, metric_filter_radio)
def panel_plots(steps_slider, speed_slider, init_x_input, init_y_input, cauchy_coeff_input, trajectory_filter_radio, metric_filter_radio):
    # Generate the trajectory based on the selected filter
    
    if trajectory_filter_radio == 'BM':
        # Generate Brownian Motion
        trajectory = brownian_motion(speed=speed_slider, n_steps=steps_slider, initial_x=init_x_input, initial_y=init_y_input)
        plot_name='Brownian Motion'
    elif trajectory_filter_radio == 'LF':
        # Generate Levy Flight
        trajectory = levy_flight(speed=speed_slider, n_steps=steps_slider, initial_x=init_x_input, initial_y=init_y_input, cauchy_coefficient=cauchy_coeff_input)
        plot_name='Levy Flight'
    elif trajectory_filter_radio == 'CRW':
        # Generate Correlated Random Walk
        trajectory = correlated_random_walk(speed=speed_slider, n_steps=steps_slider, initial_x=init_x_input, initial_y=init_y_input, cauchy_coefficient=cauchy_coeff_input)
        plot_name='Correlated Random Walk'
    else:
        trajectory = None
        plot_name = None
    
    # Plot the trajectory
    fig_3d = go.Figure()
    fig_3d.add_trace(
        go.Scatter3d(
            x = trajectory.x_pos,
            y = trajectory.y_pos,
            z = trajectory.z_pos,
            mode = 'lines',
            line=dict(color='green', width=1), 
            name = plot_name,
            showlegend = True
        )
    )
    fig_3d.update_yaxes(scaleanchor='x', scaleratio=1)
    
    # Generate metric based on the selected filter
    if metric_filter_radio == 'PL':
        # Calculate Path Length
        metric = path_length(trajectory)
        metric_name = 'Path Length'
    elif metric_filter_radio == 'MSD':
        # Calculate Mean Squared Displacement
        metric = mean_squared_displacement(trajectory)
        metric_name = 'Mean Squared Displacement'
    elif metric_filter_radio == 'TAD':
        # Calculate Total Angular Displacement
        metric = turning_angles(trajectory)
        metric_name = 'Turning Angles Distribution'
    else:
        metric = None
        metric_name = None
        
    # Plot the metric using Plotly
    fig_metric = go.Figure()
    fig_metric.add_trace(go.Scatter(
        x=metric.index,
        y=metric['y'],
        mode='lines',
        name=metric_name,
        line=dict(color='blue', width=2)
    ))
    
    fig_metric.update_yaxes(scaleanchor='x', scaleratio=1)
    
    return pn.Row(
        pn.pane.Plotly(fig_3d, sizing_mode='stretch_width'),
        pn.pane.Plotly(fig_metric, sizing_mode='stretch_width')
    )


# ----------------- CREATE DASHBOARD -----------------

# Create the layout for the first column
filtersColumn = pn.Column(
    pn.pane.Markdown("# Trajectory Plot Mode"),
    trajectory_filter_radio,
    pn.pane.Markdown("## Filters"),
    steps_slider,
    speed_slider,
    init_x_input,
    init_y_input,
    cauchy_coeff_input,
    pn.pane.Markdown("# Metric Plot Mode"),
    metric_filter_radio
)

# Plot(3D plot)
plotColumn = panel_plots

# Create the main layout with three columns
main_layout = pn.Column(filtersColumn, pn.pane.Markdown("***"), plotColumn )

# Display the panel
main_layout.servable()



BokehModel(combine_events=True, render_bundle={'docs_json': {'ff4f9709-918a-49e3-b9d2-741f56f9e60c': {'version…